# **Diplomado IA: Inteligencia Artificial II - Parte 1**. <br> Laboratorio 1: Redes Relacionales y Transformers
---
---

**Profesores:**
- Carlos Aspillaga
- Felipe del Río

**Ayudante:**
- Gabriel Sepúlveda
---
---

# **Instrucciones Generales**

El siguiente práctico se puede realizar en **parejas**. Solo uno debe realizar la entrega. El formato de entregar es el **archivo .ipynb con todas las celdas ejecutadas**. Todas las preguntas deben ser respondida en celdas de texto. No se aceptará el _output_ de una celda de código como respuesta.

**Nombre compañero 1:** COMPLETAR

**Nombre compañero 2:** COMPLETAR

**Fecha de entrega: Viernes 19 de marzo.**

El siguiente práctico cuanta con 2 secciones donde cada una contendrá 1 o más actividades a realizar. Algunas actividades correspondrán a escribir código y otras a responder preguntas. 

**Importante.** Para facilitar su ejecución, cada sección puede ser ejecutada independientemente.

Se recomienda **fuertemente** revisar las secciones donde se entrega código porque algunas actividades de código pueden reutilizar el mismo código pero con cambios en algunas líneas.



# **Agenda**

>[Diplomado IA: Inteligencia Artificial II - Parte 1.  Laboratorio 1: Redes Relacionales y Transformers](#scrollTo=tHopPtVaNF1K)

>[Instrucciones Generales](#scrollTo=uIdAKAdELPSl)

>[Agenda](#scrollTo=kEloa5uXLIPK)

>[Parte I: Inspeccionando el encoder de un Transformer](#scrollTo=ZS3cYFT2TWB9)

>>[Preámbulo](#scrollTo=0dBeU4b818s4)

>>[Representación del Input para BERT](#scrollTo=WlA3k2R7gPWV)

>>>[Actividad 1](#scrollTo=bl9cd84BWtuK)

>>[NER](#scrollTo=ukXvd54RjC_4)

>>[Visualización de Atenciones](#scrollTo=M7aoVKrWlLbp)

>>>[Head View](#scrollTo=Z4COgQK0BAsp)

>>>[Model View](#scrollTo=J9e2nT-FD_kH)

>>>[Neuron View](#scrollTo=in3biNv0pJV_)

>>>>[Actividad 2](#scrollTo=7krb_CeKqANH)

>[Parte II: Inspeccionando el decoder de un Transformer](#scrollTo=6Yf6uthYE2pA)

>>[Preámbulo](#scrollTo=wJdA60kAbgHP)

>>[Traducción EN-DE](#scrollTo=oyk-K3pjXrwX)

>>>[Actividad 3](#scrollTo=flpPAFaJfm5H)



# Parte I: Inspeccionando el encoder de un Transformer

En este laboratorio exploraremos un modelo de Transformer, BETO, el cual está basado en BERT y preentrenado en un corpus en español. Utilizaremos un modelo que ya fue refinado para la tarea de NER en español. 

**NER (*Named Entity Recognition*)**, es una tarea que consiste en encontrar y detectar entidades nombradas (personas, organizaciones, etc.) dentro de un texto, además de clasificar a que tipo corresponde. Es sumemente útil para identificar que entidades están presentes en nuestros textos y poder un análisis más profundo en base a ellos.


## Preámbulo

Primero debemos descargar, instalar e importar las distintas librerías que utilizaremos para este laboratorio.

In [ ]:
import sys
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex transformers -q
!pip install -qr bertviz_repo/requirements.txt
!pip install -q urllib3==1.25.10

In [ ]:
import string

import IPython
from bertviz import head_view, model_view
from bertviz.neuron_view import show
import spacy
from spacy.vocab import Vocab
from spacy.tokens import Doc, Span
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import BertTokenizer, BertModel

También crearemos algunas funciones auxiliares para el laboratorio.

In [ ]:
# Funciones auxiliares para la visualización de los resultados del modelo para NER

LABEL_LIST = [
    "B-LOC",    # Beginning of a location right after another location
    "B-MISC",   # Beginning of a miscellaneous entity right after another miscellaneous entity
    "B-ORG",    # Beginning of an organisation right after another organisation
    "B-PER",    # Beginning of a person's name right after another person's name
    "I-LOC",    # Location
    "I-MISC",   # Miscellaneous entity
    "I-ORG",    # Organisation
    "I-PER",    # Person's name
    "O"         # Outside of a named entity
]

# Construimos el Doc para uso de Spacy
def build_doc(tokens, spaces, vocab):
    tokens = [token.replace('##', '') for token in tokens] # Remove ## start in word pieces
    vocab = Vocab(strings=vocab)
    doc = Doc(vocab=vocab, words=tokens, spaces=spaces)
    return doc

# Obtenemos la lista de `spaces`, con `True` para los tokens que están seguidos de espacios
def get_spaces(tokens):
    spaces = []
    for i, token in enumerate(tokens):
        if i + 1 == len(tokens):
            spaces.append(False)
            break

        next_token = tokens[i+1]
        if next_token in string.punctuation:
            spaces.append(False)
            continue
        if next_token.startswith('##'):
            spaces.append(False)
            continue
        
        spaces.append(True)

    return spaces

# Transformar de una lista de entidades a un diccionarion con las entidades
# interesantes y sus ubicaciones
def get_entity_locs(entities):
    def gettype(ent):
        return ent.replace('B-', '').replace('I-', '')

    ents = []
    start, end = 0, 0
    type_ = ''
    for i, ent in enumerate(entities):
        if ent == 'O':
            if type_:
                ents.append((start, end, type_))
            start, end = 0, 0
            type_ = ''

        if ent.startswith('B-'):
            if type_:
                ents.append((start, end, type_))
            start, end = i, i
            type_ = gettype(ent)

        if type_ and ent.startswith('I-') and gettype(ent) == type_:
            end = i

        if i + 1 == len(entities) and type_:
            ents.append((start, end, type_))
            
    return ents

# Añadimos las entidades al objeto Doc
def add_entities(doc, entity_locs):
    doc_ents = []
    for start, end, type_ in entity_locs:
        span = Span(doc, start=start, end=end+1, label=type_)
        doc_ents.append(span)

    doc.ents = doc_ents
    return doc

# Integramos el pipeline completo para visualizar las entidades
def build_for_vis(tokens, predictions, special_tokens=None):
    special_tokens = [] if special_tokens is None else special_tokens

    ents = [LABEL_LIST[prediction] for token, prediction in zip(tokens, predictions) if token not in special_tokens]
    tokens = [token for token in tokens if token not in special_tokens]

    spaces = get_spaces(tokens)
    vocab = list(tokenizer.vocab.keys())
    doc = build_doc(tokens, spaces, vocab=vocab)

    entity_locs = get_entity_locs(ents)
    doc = add_entities(doc, entity_locs)
    return doc

# Funciones auxiliares para la visualización de atenciones

def call_html(view='head'):
    assert view in ['head', 'model', 'neuron']
    d3_version = {
        'head': '3.5.8', 'model': '5.7.0', 'neuron': '5.7.0'
    }[view]
    load_libs_html = f'''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
            requirejs.config({{
                paths: {{
                base: '/static/base',
                "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/{d3_version}/d3.min",
                jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
                }},
            }});
            </script>
            '''
    display(IPython.core.display.HTML(load_libs_html))

## Representación del Input para BERT

Primero que todo, creamos el modelo y tokenizador a utilizar. Como vimos en clases, un modelo del tipo BERT requiere formatear la entrada agregando tokens especiales que cumplen algunas funciones. Para lograr este formato de forma sencilla, utilizaremos el tokenizador correspondiente.

In [ ]:
device = torch.device('cuda')

model_type = 'bert'
model_version = 'mrm8488/bert-spanish-cased-finetuned-ner' # Pesos a utilizar

tokenizer = AutoTokenizer.from_pretrained(model_version)
model = AutoModelForTokenClassification.from_pretrained(model_version, output_attentions=True)
model.to(device)

special_tokens = tokenizer.special_tokens_map.values()

In [ ]:
text = 'Eduardo Vargas le metió un gol a España en el mundial de Brasil.'

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
inputs = tokenizer(text, return_tensors="pt")
inputs.to(device)
None

A continuación utilizaremos el tokenizador que nos provee la librería Transformers para entender mejor como se construye el input para un modelo basado en BERT.

Podemos observar cual es el resultado de agregar estos tokens a nuestro texto. Para esto usamos el *hack*, de codificar y luego decodificar nuestro texto.

In [ ]:
tokenizer.decode(tokenizer.encode(text))

Sabemos que BERT está pensado para utilizar varias frases en el input, como por ejemplo para tareas de pregunta-respuesta podemos ver el resultado de la misma forma, entregandole el segundo texto como input al tokenizador.

In [ ]:
text2 = 'Y Alexis Sánchez a Brasil en octavos, en Belo Horizonte'
tokenizer.decode(tokenizer.encode(text, text2))

También, podemos ver la forma en que se hace tokenización a nivel de sub-palabras, es decir que existen tokens que corresponden a partes de palabras en vez de la palabra completa.

In [ ]:
tokens

El tokenizador nos ayuda también a preparar los otros inputs que entrarán al modelo, como lo son por ejemplo los *segment embeddings*. A continuación se muestran tanto para una frase como para dos.



![](https://miro.medium.com/max/1000/1*EKzyGf_l0e57XN491_YAyg.png)

In [ ]:
sample_input = tokenizer(text, return_tensors="pt")
sample_input['token_type_ids']

In [ ]:
sample_input2 = tokenizer(text, text2, return_tensors="pt")
sample_input2['token_type_ids']

### Actividad 1

Ejecute las celdas correspondientes y vaya contestando las preguntas asociadas.

En base a las celdas anteriores, **¿qué significa los valores de 1 obtenidos cuando inspeccionamos los `token_type_ids`?**

In [ ]:
R = 'Responda brevemente.' # @param {type: "string"}

Veamos los tokenes especiales que utilizará nuestro tokenizador.

In [ ]:
print(', '.join(special_tokens))

In [ ]:
SEP = '¿Cuál es la función del token especial [SEP]?' #@param {type:"string"}
CLS = '¿Cuál es la función del token especial [CLS]?' #@param {type:"string"}
MASK = '¿Cuál es la función del token especial [MASK]?' #@param {type:"string"}

Considere una situación similar a la actual en la cuál utilizará un modelo que ya está entrenado, para hacer inferencias en producción. 

**¿Cuál de los siguientes tokenes especiales nunca esperaría ver en el input?**


In [ ]:
R ='[UNK]' #@param ["[UNK]", "[SEP]", "[CLS]", "[MASK]"]

Considere nuestro modelo, como un modelo del tipo BERT. 

**¿Por qué no se muestra un token del tipo [EOS] (end-of-string) como token especial?**


In [ ]:
R = 'Responda brevemente' #@param {type: "string"}

**¿En qué se diferencia el modelo BETO de BERT?**

In [ ]:
R = 'En sus arquitecturas.' #@param ["En sus arquitecturas.", "En los datos de entrenamiento.", "En el algoritmo de optimización.", "En la tarea usada para el pre-entrenamiento (modelo de leguaje)."]

## NER

Ahora utilicemos el modelo que creamos para la tarea de NER y veamos las predicciones que realiza

In [ ]:
outputs = model(**inputs)[0]
predictions = torch.argmax(outputs, dim=2)
predictions = predictions[0].tolist()

In [ ]:
labeled_text = build_for_vis(tokens, predictions, special_tokens=special_tokens)

In [ ]:
from spacy import displacy

displacy.render(labeled_text, style="ent", jupyter=True)

Puede utilizar el código a continuación para obtener una visualización en base al texto que ud. desee. Para esto debe asignarlo a la variable `texto`.

In [ ]:
texto = '' #@param {type:"string"}

if texto:
    tokens = tokenizer.tokenize(
        tokenizer.decode(tokenizer.encode(texto)))
    inputs = tokenizer(texto, return_tensors="pt")
    inputs.to(device)

    outputs = model(**inputs)[0]
    predictions = torch.argmax(outputs, dim=2)
    predictions = predictions[0].tolist()

    labeled_text = build_for_vis(
        tokens, predictions, special_tokens=special_tokens)
    displacy.render(labeled_text, style="ent", jupyter=True)

## Visualización de Atenciones

Como sabemos, la arquitectura de un modelo Transformer está construída en gran parte sobre atenciones. Esto nos da la posibilidad de poder visualizar estas atenciones para poder hacernos una idea de lo que está pasando dentro del modelo mismo. Para generar las visualizaciones nos ayudará el repositiorio [BertViz](https://github.com/jessevig/bertviz).

**Importante.** Es posible que las visualizaciones a continuación no se carguen correctamente la primera vez que se ejecuta la celda, en dicho caso intenten ejecutar nuevamente la celda. 

### Head View

Primero usamos la visualización del tipo `head_view` para ver los patrones de atención de una o más *heads* para una capa dada. 

En esta visualización podemos ver la representación de la palabra que está siendo actualizada (*query*) a la **izquierda**, mientras que las palabras a las cuales se le pone atención (*key, value*) a la **derecha** de la visualización.

Con el *dropdown* podemos cambiar la capa en la cual nos fijamos y cada color significa una *head* distinta del modelo.

In [ ]:
# Head View
texto = 'Eduardo Vargas le metió un gol a España en el mundial de Brasil.'

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(texto)))
inputs = tokenizer(texto, return_tensors="pt")
inputs.to(device)

token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']

attention = model(**inputs)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html(view='head')

head_view(attention, tokens)

Puede utilizar el código a continuación para obtener una visualización en base al texto que ud. desee. Para esto debe asignarlo a la variable `texto`.

In [ ]:
# Head View
texto = '' #@param {type:"string"}

if texto:
    tokens = tokenizer.tokenize(
        tokenizer.decode(tokenizer.encode(texto)))
    inputs = tokenizer(texto, return_tensors="pt")
    inputs.to(device)

    token_type_ids = inputs['token_type_ids']
    input_ids = inputs['input_ids']

    attention = model(**inputs)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    call_html(view='head')

    head_view(attention, tokens)

### Model View

A continuación  tenemos una visualización muy similar a  la anterior, sin embargo esta muestra cada capa y head del modelo por separado.

Cada fila está asociada a una capa del modelo (en orden creciente) mientras que cada columna corresponde a una *head* distinta en dicha capa.

In [ ]:
texto = 'Eduardo Vargas le metió un gol a España en el mundial de Brasil.'

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(texto)))
inputs = tokenizer(texto, return_tensors="pt")
inputs.to(device)

token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(**inputs)[-1]
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)
call_html(view='model')

model_view(attention, tokens)

A continuación puede utilizar el código a continuación para obtener una visualización en base al texto que ud. desee. Para esto debe asignarlo a la variable `texto`.

In [ ]:
texto = '' #@param {type:"string"}

if texto:
    tokens = tokenizer.tokenize(
        tokenizer.decode(tokenizer.encode(texto)))
    inputs = tokenizer(texto, return_tensors="pt")
    inputs.to(device)

    token_type_ids = inputs['token_type_ids']
    input_ids = inputs['input_ids']
    attention = model(**inputs)[-1]
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    call_html(view='model')

    model_view(attention, tokens)

### Neuron View

A continuación tenemos una visualización muy similar a las anteriores, principalmente a la primera. Sin embargo, en esta podemos visualizar de mucho mejor forma las activaciones que están generando para cada una de las representaciones y como estas afectan la atención.

Este gráfico se interpreta de la misma manera que el primero (*Head View*), sin embargo podemos hacer click en las palabras de la izquierda para visualizar el cálculo de cada atención.

**Importante.** Este modelo trabaja con una versión modificada de la librería Transformer que hemos utilizada (notar los `import`'s a continuación), es por eso que las frases a utilizar son en inglés pues solo contamos con los modelos pre-entrenados base.

In [ ]:
from bertviz.transformers_neuron_view import BertModel as VizBertModel, BertTokenizer as VizBertTokenizer
from bertviz.neuron_view import show

In [ ]:
# Ojo: Puede haber problemas con la visualziación si es que las oraciones son muy largas.
sentence_a = "Alexis scored against Brazil in the World Cup."
sentence_b = "Pinilla's shot struck the crossbar in that match."

In [ ]:
nv_model_type = 'bert'
nv_model_version = 'bert-base-uncased'

do_lower_case = 'uncased' in nv_model_version
tokenizer = VizBertTokenizer.from_pretrained(nv_model_version, do_lower_case=do_lower_case)
model = VizBertModel.from_pretrained(nv_model_version)
call_html(view='neuron')
show(model, nv_model_type, tokenizer, sentence_a, sentence_b)

#### Actividad 2

Descomente el código a continuación y elija una versión de BERT para probar la misma visualización que la obtenida anteriormente. Para esto debe definir correctamente la variable `nv_model_version`. 

Basta con la ejecución para la actividad. Debe ser una versión distinta a la ya utilizada, `'bert-base-uncased'`.

**Importante:** Por limitaciones del sistema, no elijan modelos muy pesados para la visualización pues probablemente se caerán, principalmente eviten los modelos con `large` en el nombre.

In [ ]:
# nv_model_type = 'bert'
# nv_model_version = 'bert-base-cased'

# do_lower_case = 'uncased' in nv_model_version
# tokenizer = VizBertTokenizer.from_pretrained(nv_model_version, do_lower_case=do_lower_case)
# model = VizBertModel.from_pretrained(nv_model_version)
# call_html(view='neuron')
# show(model, nv_model_type, tokenizer, sentence_a, sentence_b)

# Parte II: Inspeccionando el decoder de un Transformer

En esta parte de la actividad inspeccionaremos la atención cruzada que realiza el decoder sobre el encoder. Esta atención nos dará una mejor idea de que está usando el modelo para generar texto en el decoder. 



## Preámbulo

Primero debemos instalar las dependencias que usaremos durante la actividad y los checkpoints del modelo. Utilizaremos un modelo pre-entrenado que nos [provee OpenNMT](https://opennmt.net/Models-py/). 

In [ ]:
!pip install git+https://github.com/OpenNMT/OpenNMT-py.git -q
!mkdir -p checkpoints
!wget -P checkpoints https://s3.amazonaws.com/opennmt-models/transformer-ende-wmt-pyOnmt.tar.gz
!tar xzf checkpoints/transformer-ende-wmt-pyOnmt.tar.gz -C checkpoints

También descargaremos los datos que se utilizarán, en este caso el dataset entre inglés y alemán [WMT](http://www.statmt.org/wmt14/translation-task.html]).

In [ ]:
!mkdir -p data
!wget -P data https://s3.amazonaws.com/opennmt-trainingdata/wmt_ende_sp.tar.gz
!tar xzf data/wmt_ende_sp.tar.gz -C data

También importamos nuestras dependencias a nuestro entorno de trabajo.

In [ ]:
from collections import defaultdict, Counter
import random

import torch
from torch import nn
from tqdm import tqdm_notebook as tqdm

import onmt
from onmt.inputters.inputter import _load_vocab, _build_fields_vocab, get_fields
from onmt.model_builder import build_model
from onmt.models.model_saver import load_checkpoint
from onmt.translate import GNMTGlobalScorer, Translator, TranslationBuilder
from onmt.opts import dynamic_prepare_opts, translate_opts
from onmt.utils.parse import ArgumentParser

import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

Debemos crear el archivo de configuración que utilizaremos con el modelo. Este archivo de configuración es una adaptación al indicado en la [documentación de OpenNMT](http://opennmt.net/OpenNMT-py/FAQ.html#how-do-i-use-the-transformer-model-do-you-support-multi-gpu).

In [ ]:
%%writefile checkpoints/config.yaml

src_vocab: data/wmtende.vocab

# Corpus opts:
data:
    train:
        path_src: data/train.de
        path_tgt: data/train.en

# General opts
save_model: foo
save_checkpoint_steps: 10000
valid_steps: 10000
train_steps: 200000

# Batching
queue_size: 10000
bucket_size: 32768
world_size: 4
gpu_ranks: [0, 1, 2, 3]
batch_type: "tokens"
batch_size: 4096
valid_batch_size: 8
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp32"
optim: "adam"
learning_rate: 2
warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
rnn_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]

Finalmente definimos algunas funciones que nos ayudarán a simplificar el código de la actividad.

In [ ]:
# Obtención del checkpoint en base a su path
def get_checkpoint_data(checkpoint_path):
    checkpoint = load_checkpoint(checkpoint_path)
    fields = checkpoint['vocab']
    opts = checkpoint['opt']
    return checkpoint, fields, opts

# Actualización de la configuración por defecto en base a las del checkpoint
def update_opts(opts, config_path):
    parser = ArgumentParser(description='translate.py')
    dynamic_prepare_opts(parser, build_vocab_only=False)
    translate_opts(parser)

    base_args = ['-config', config_path,
                 '-out_file', './out.txt',
                 '-model', 'checkpoints/averaged-10-epoch.pt',
                 '-src', 'data/valid.en',
                ]

    opts, unknown = parser.parse_known_args(base_args, namespace=opts)

    model_opt = ArgumentParser.ckpt_model_opts(opts)
    ArgumentParser.update_model_opts(model_opt)
    ArgumentParser.validate_model_opts(model_opt)

    opts.out_file = './out.txt'
    return opts, model_opt


def build_dataset(src_path, tgt_path):
    src_data = {"reader": onmt.inputters.str2reader["text"].from_opt(opts), "data": src_path}
    tgt_data = {"reader": onmt.inputters.str2reader["text"].from_opt(opts), "data": tgt_path}
    _readers, _data = onmt.inputters.Dataset.config(
        [('src', src_data), ('tgt', tgt_data)])

    dataset = onmt.inputters.Dataset(
        fields, readers=_readers, data=_data,
        sort_key=onmt.inputters.str2sortkey["text"])

    return dataset

def build_translator(opts, model, dataset, fields, tgt_path):
    scorer = onmt.translate.GNMTGlobalScorer.from_opt(opts)

    translator = Translator.from_opt(
        model,
        fields,
        opts,
        model_opt,
        global_scorer=scorer,
        out_file=opts.out_file,
        report_align=False,
        report_score=True
    )

    builder = onmt.translate.TranslationBuilder(
        dataset, fields, opts.n_best, opts.replace_unk, tgt_path,
        opts.phrase_table
    )

    return translator, scorer, builder

# Visualización de atenciones
def draw(data, x, y, ax):
    seaborn.heatmap(data, 
                    xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0, 
                    cbar=False, ax=ax)

## Traducción EN-DE

Utilizaremos un modelo Transformer para generar traducciones desde el inglés al alemán. 

Al generar estas traducciones, más que enfocarnos en ella nos interesa visualizar las atenciones que pone el modelo, en este caso enfoccandonos a las atenciones que el decoder del transformer pone sobre el resultado del encoder. En la literatura esta atención a veces se le llama **cross-attention**.


In [ ]:
checkpoint_path = 'checkpoints/averaged-10-epoch.pt'
config_path = 'checkpoints/config.yaml'

checkpoint, fields, opts = get_checkpoint_data(checkpoint_path)
opts, model_opt = update_opts(opts, config_path)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Como siempre, primero creamos el dataset y el modelo. También precargamos los pesos al modelo. Como estamos usando OpenNMT, creamos también un objeto del tipo `Translator` que nos ayudará a generar las traducciones de forma más limpia. 

In [ ]:
src = 'data/valid.en'
tgt = 'data/valid.de'

dataset = build_dataset(src, tgt)
data_iter = onmt.inputters.OrderedIterator(
                dataset=dataset,
                device=device,
                batch_size=16,
                train=False,
                sort=False,
                sort_within_batch=True,
                shuffle=True,
            )

In [ ]:
model = build_model(model_opt, opts, fields, checkpoint)
model.to(device)

translator, scorer, builder = build_translator(opts, model, dataset, fields, tgt)

Ahora, utilizando un batch aleatorio del set de validación. Veremos la traducción generada y mostraremos las atenciones que utiliza el modelo sobre el encoder.

**Importante.** Por limitaciones de la API de OpenNMT, solo se visualiza el primer head de las atenciones en la última capa del Transformer.

In [ ]:
for batch in data_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=dataset.src_vocabs,
        attn_debug=True)
    translations = builder.from_batch(trans_batch)
    break

In [ ]:
# Seleccionamos aleatoriamente alguno de los elementos del batch
translation = random.choice(translations)
attns = translation.attns[0]
attns = attns.cpu() 

x_len = len(translation.src_raw)
attns = attns[:,:x_len]

# Agregamos el token de término a la predicción 
tgt = translation.pred_sents[0] + ['[EOS]']

fig, axs = plt.subplots(1,1, figsize=(18, 10))
draw(attns, translation.src_raw, tgt, ax=axs)
plt.show()

### Actividad 3

Responda las preguntas a continuación.

En el archivo de configuración tenemos la siguiente información:

```
...
enc_layers: 6
dec_layers: 6
heads: 8
...
```
Al generar el gráfico anterior, solo lo estamos visualizando las cross attentions para una capa y una "head" del modelo. **¿Cuántos gráficos tenedremos si es que pudiesemos visualizar todas las cross-attentions?**

In [ ]:
R = 0 # @param{type: "number"}

El decoder del Transformer, no solo utiliza este tipo de atenciones, también utiliza las del tipo self-attention. Si estuvieramos haciendo decoding de una traducción en el paso T=5, es decir generando el 5to token de salida , tal y como se muestra en la siguiente imágen.

![](https://www.guru99.com/images/1/111318_0848_seq2seqSequ1.png)

Si visualizaramos las atenciones de self-attention **¿Qué dimensiones tendría la matriz mostrada?**

In [ ]:
R = '0x0' # @param {type: "string"}

**Responda si la siguiente afirmación es verdadera o falsa.** 

BERT **no** utiliza cross-attentions.


In [ ]:
R = '' #@param ["", "Verdadero", "Falso"]

BETO **sí** utiliza cross-attentions.

In [ ]:
R = '' #@param ["", "Verdadero", "Falso"]

En el decoder de un transformer (como lo vimos en clase) no es necesario enmascarar "el futuro".

In [ ]:
R = '' #@param ["", "Verdadero", "Falso"]